In [20]:
# 定义批量删除的查询
batch_size = 500000
while True:
    query = f"""
    MATCH (p:trajectory_point)
    WITH p LIMIT {batch_size}
    DETACH DELETE p
    RETURN COUNT(p) AS deleted_count
    """
    
    # 执行批量删除操作
    result = g.run(query).data()
    
    # 如果没有节点被删除，结束循环
    if result[0]['deleted_count'] == 0:
        break

    print(f"Deleted {result[0]['deleted_count']} nodes in this batch.")

In [21]:
# 删除所有 'next' 关系
g.run("MATCH ()-[r:next]->() DELETE r")

(No data)

In [22]:
cypher = 'MATCH (n:trajectory_point) RETURN count(n) AS total_trajectory_point_nodes'
g.run(cypher)

total_trajectory_point_nodes
0


In [47]:
selected_files = dir_xls[4000:]
print("selected_files第一个元素:", selected_files[0])
print("selected_files最后一个元素:", selected_files[-1])

selected_files第一个元素: F:\paper2\出租车原始数据\part2\12202.csv
selected_files最后一个元素: F:\paper2\出租车原始数据\part2\13831.csv


In [48]:
selected_files = dir_xls[4000:]
dir_xls = selected_files
print("dir_xls第一个元素:", dir_xls[0])
print("dir_xls最后一个元素:", dir_xls[-1])

dir_xls第一个元素: F:\paper2\出租车原始数据\part2\12202.csv
dir_xls最后一个元素: F:\paper2\出租车原始数据\part2\13831.csv


In [2]:
import os
os.environ['PROJ_LIB'] = r'D:\Anaconda\envs\TrKG\Library\share\proj'
os.environ['GDAL_DATA'] = r'CD:\Anaconda\envs\TrKG\Library\share'
import pandas as pd
import geopandas as gpd
from functools import reduce
import numpy as np

def get_file(folder_path):  #获取同一文件夹下所有的文件路径
    dir_file = os.listdir(folder_path)
    for path in dir_file:
        if path[-3:] == 'csv' or path[-4:] == 'xlsx' or path[-3:] == 'xls':
            whole_path = (folder_path + "\\" + '{}').format(path)
            dir_xls.append(whole_path)
    return dir_xls
def get_name(folder_path):  #获取同一文件夹下所有的文件路径
    dir_file = os.listdir(folder_path)
    for path in dir_file:
        if path[-3:] == 'csv' or path[-4:] == 'xlsx' or path[-3:] == 'xls':
            file_name_without_extension = os.path.splitext(path)[0]  # 获取文件名的主干，不包含扩展名
            folder_name.append(file_name_without_extension)
    return folder_name

In [ ]:
gdf_honeycomb = r"F:\paper2\result\dz_honeycomb_125.shp"
gdf_honeycomb = gpd.read_file(gdf_honeycomb)
print(gdf_honeycomb.crs)
# 读取道路网络的 Shapefile
road = r"E:\paper2\S-上海-SH\01-SHP\01-市域数据（❤❤❤）\2023（❤❤❤）\上海市_主干道.shp"
road_gdf = gpd.read_file(road)
road_gdf = road_gdf.to_crs(gdf_honeycomb.crs)
print(road_gdf.crs)

In [ ]:
folder_path = r'F:\paper2\出租车原始数据'
file_list = os.listdir(folder_path)
full_paths = [os.path.join(folder_path, f) for f in file_list]
print(full_paths)

In [4]:
# 生成时间序列，5分钟为一个时间切片
helper = pd.date_range(start='2015/4/1 0:00:00', end='2015/5/1 0:00:00', freq='5min', name='datetime')
helper = pd.DataFrame(helper, columns=['datetime'])

# 对 helper 的 'datetime' 列做调整，除第一个时间点之外的所有时间向后调整1微秒
helper['datetime'][1:] = helper['datetime'][1:] + pd.Timedelta(microseconds=1)
# 查看 helper 中每列的数据类型
print(helper.dtypes)
helper

datetime    datetime64[ns]
dtype: object


,datetime
0,2015-04-01 00:00:00.000000
1,2015-04-01 00:05:00.000001
2,2015-04-01 00:10:00.000001
3,2015-04-01 00:15:00.000001
4,2015-04-01 00:20:00.000001
...,...
8636,2015-04-30 23:40:00.000001
8637,2015-04-30 23:45:00.000001
8638,2015-04-30 23:50:00.000001
8639,2015-04-30 23:55:00.000001


In [1]:
dir_xls = []

for folder_path in full_paths:
    # 获取文件和文件名
    dir_xls = get_file(folder_path)


# 按文件名进行自然排序（1, 2, 10 的顺序）
dir_xls = sorted(dir_xls, key=lambda x: int(os.path.basename(x).split('.')[0]))
print(len(dir_xls))
print(dir_xls)

NameError: name 'full_paths' is not defined

In [42]:
from py2neo import Node, Relationship, Graph, NodeMatcher, RelationshipMatcher, Subgraph
import os
import json

g = Graph('bolt://localhost:7687',auth=('neo4j','your password'), name = 'trkg')
node_matcher = NodeMatcher(g)
relation_matcher = RelationshipMatcher(g)

In [8]:
# 缓存 honeycomb 节点以加快查询速度
honeycomb_nodes = {honeycomb['name']: honeycomb for honeycomb in node_matcher.match("honeycomb")}

In [49]:
from shapely.wkt import loads as wkt_loads  # 用于解析WKT格式的几何数据
from shapely.geometry import Point  # 用于创建点对象
from datetime import datetime  # 用于时间运算
from tqdm.notebook import tqdm
from shapely.errors import GEOSException  # 导入 GEOSException
import pandas as pd
import geopandas as gpd
from py2neo import Graph, Node, Relationship, Subgraph  # 导入 py2neo 中的图、节点和关系对象

BATCH_SIZE = 200000  # 批次大小
trajectory_point_ls = []  # 用于存储待提交的轨迹点
relationship_ls = []  # 用于存储待提交的关系
# 计算总的时间切片数
total_slices = len(dir_xls) * (len(helper) - 1)

# 创建一个覆盖所有时间切片的进度条
with tqdm(total=total_slices, desc="Processing time slices") as time_slice_pbar:

    for file_path in dir_xls:
        
        # if file_path == 'F:\\paper2\\出租车原始数据\\part1\\2.csv':
        #     break
            
        df_each = pd.read_csv(file_path, usecols=[0, 1, 2, 3, 4],
                              names=['vehicle', 'passenger_', 'datetime', 'longitude', 'latitude'], header=None)

        # 将 'longitude' 和 'latitude' 列转换为几何数据，并创建 GeoDataFrame
        trajectory = gpd.GeoDataFrame(df_each,
                                      geometry=gpd.points_from_xy(df_each['longitude'], df_each['latitude']),
                                      crs='EPSG:4326')
        trajectory = trajectory.to_crs(gdf_honeycomb.crs)

        trajectory['pre_code_helper'] = range(len(trajectory))
        minx, miny, maxx, maxy = trajectory.total_bounds

        try:
            # 根据trajectory的空间范围裁剪road_gdf
            road_gdf_clipped = road_gdf.cx[minx:maxx, miny:maxy][['geometry']]  # 只保留 geometry 列以减少数据量
        except GEOSException as e:
            print(f"GEOSException encountered in file {file_path}: {e}, skipping this iteration.")
            continue

        # 批量空间连接操作，找到最近的道路
        trajectory_matched = gpd.sjoin_nearest(trajectory, road_gdf_clipped, how='inner', max_distance=30, distance_col="distance")
        # 只保留 `trajectory` 的字段
        trajectory_matched = trajectory_matched[trajectory.columns.tolist()]
        # 使用批量化的空间连接操作将 `gdf_honeycomb` 的 `FID` 列加入到 `trajectory_matched` 中
        trajectory_matched = gpd.sjoin(trajectory_matched, gdf_honeycomb[['geometry']], how='inner', predicate='within')
        
        if not trajectory_matched.empty:
            trajectory_matched = trajectory_matched.drop_duplicates(subset=['pre_code_helper'], keep='first')
            trajectory_matched['datetime_helper'] = pd.to_datetime(trajectory_matched['datetime'], format="%Y-%m-%d %H:%M:%S")
            trajectory_matched = trajectory_matched.set_index('datetime_helper').sort_index()
            trajectory_matched['pre_code'] = range(len(trajectory_matched))
            # print(f"trajectory_matched\n {trajectory_matched}")

            vehicle = trajectory_matched['vehicle'].iat[0]
            start, end = 0, 1
            batch_num = 1

            for k in range(len(helper) - 1):
                
                # if k != 1:
                #     continue
                    
                start_date = helper.loc[start, 'datetime']
                end_date = helper.loc[end, 'datetime']
                trajectory_matched_slice = trajectory_matched.loc[start_date:end_date]

                if not trajectory_matched_slice.empty:
                    batch = f'batch_{batch_num}'

                    temp_trajectory_point_ls = []
                    temp_relationship_ls = []

                    for i in range(len(trajectory_matched_slice)):
                        point = trajectory_matched_slice.iloc[i]
                        trajectory_node = Node('trajectory_point',
                                               name=int(point['pre_code']),
                                               passenger=str(point['passenger_']),
                                               datetime=str(point['datetime']),
                                               geometry=str(point['geometry']),
                                               vehicle=int(vehicle),
                                               batch=batch)
                        temp_trajectory_point_ls.append(trajectory_node)

                        # 获取匹配的多边形的 FID，直接从提前计算好的列中获取
                        honeycomb_idx = point['index_right']
                        # print(f"honeycomb_idx\n {honeycomb_idx}")
                        honeycomb = honeycomb_nodes.get(honeycomb_idx)  # 从缓存中查找 `honeycomb` 节点
                        # 构建 'located_in' 关系
                        located_in_rel = Relationship(trajectory_node, "located_in", honeycomb)
                        temp_relationship_ls.append(located_in_rel)
                        
                        # 如果当前点不是第一个点，创建与前一个点的 "next" 关系
                        if i > 0:
                            prev_node = temp_trajectory_point_ls[i - 1]
                            next_relationship = Relationship(prev_node, "next", trajectory_node)
                            temp_relationship_ls.append(next_relationship)

                    # 批量添加节点和关系
                    trajectory_point_ls.extend(temp_trajectory_point_ls)
                    relationship_ls.extend(temp_relationship_ls)

                    # 检查是否达到批次大小并提交，仅依据 trajectory_point_ls 的长度
                    if len(trajectory_point_ls) >= BATCH_SIZE:
                        # 批量提交所有节点和关系到数据库
                        subgraph = Subgraph(nodes=trajectory_point_ls, relationships=relationship_ls)
                        tx = g.begin()
                        tx.create(subgraph)
                        g.commit(tx)

                        # 清空临时列表
                        trajectory_point_ls.clear()
                        relationship_ls.clear()

                start += 1
                end += 1
                batch_num += 1

        # 更新进度条：每处理完一个文件更新一次
        time_slice_pbar.update(len(helper) - 1)

# 提交剩余的节点和关系（外层提交）
if trajectory_point_ls:
    subgraph = Subgraph(nodes=trajectory_point_ls, relationships=relationship_ls)
    tx = g.begin()
    tx.create(subgraph)
    g.commit(tx)

    trajectory_point_ls.clear()
    relationship_ls.clear()

Processing time slices:   0%|          | 0/14083200 [00:00<?, ?it/s]

GEOSException encountered in file F:\paper2\出租车原始数据\part2\13371.csv: IllegalArgumentException: Points of LinearRing do not form a closed linestring, skipping this iteration.
GEOSException encountered in file F:\paper2\出租车原始数据\part2\13811.csv: IllegalArgumentException: Points of LinearRing do not form a closed linestring, skipping this iteration.


In [11]:
from concurrent.futures import ThreadPoolExecutor
from shapely.wkt import loads as wkt_loads  # 用于解析WKT格式的几何数据
from shapely.geometry import Point  # 用于创建点对象
from datetime import datetime  # 用于时间运算
from tqdm.notebook import tqdm
import threading  # 为线程锁引入 threading 模块
from shapely.errors import GEOSException  # 导入 GEOSException


trajectory_point_ls = []  # 用于存储待提交的轨迹点
relationship_ls = []  # 用于存储待提交的关系
lock = threading.Lock()  # 创建线程锁

# 计算总的时间切片数
total_slices = len(dir_xls) * (len(helper) - 1)

# 创建一个覆盖所有时间切片的进度条
with tqdm(total=total_slices, desc="Processing time slices") as time_slice_pbar:

    def process_slice(trajectory_matched, helper, start, end, batch_num):
        temp_trajectory_point_ls = []
        temp_relationship_ls = []

        start_date = helper.loc[start, 'datetime']
        end_date = helper.loc[end, 'datetime']
        trajectory_matched_slice = trajectory_matched.loc[start_date:end_date]

        if not trajectory_matched_slice.empty:
            
            # print('trajectory_matched_slice\n', trajectory_matched_slice[['pre_code', 'vehicle']])
            
            batch = f'batch_{batch_num}'

            for i in range(len(trajectory_matched_slice)):
                point = trajectory_matched_slice.iloc[i]
                trajectory_node = Node('trajectory_point',
                                       name=int(point['pre_code']),
                                       passenger=str(point['passenger_']),
                                       datetime=str(point['datetime']),
                                       longitude=float(point['longitude']),
                                       latitude=float(point['latitude']),
                                       geometry=str(point['geometry']),
                                       vehicle=int(vehicle),
                                       batch=batch)
                temp_trajectory_point_ls.append(trajectory_node)

                # 如果不是第一个点，创建与前一个点的关系
                if i > 0:
                    prev_node = temp_trajectory_point_ls[i - 1]

                    # # 使用Shapely解析WKT格式的几何数据
                    # prev_geometry = wkt_loads(prev_node['geometry'])
                    # curr_geometry = wkt_loads(trajectory_node['geometry'])

                    # # 计算两个几何点之间的距离
                    # distance = round(prev_geometry.distance(curr_geometry), 2)

                    # 提取时间，计算时间差（秒）
                    # prev_time = datetime.strptime(prev_node['datetime'], '%Y-%m-%d %H:%M:%S')
                    # curr_time = datetime.strptime(trajectory_node['datetime'], '%Y-%m-%d %H:%M:%S')
                    # time_interval = (curr_time - prev_time).total_seconds()  # 保留时间差为秒

                    # # 计算速度 (单位：米/秒)
                    # if time_interval > 0:  # 避免除以零的错误
                    #     velocity = round(distance / time_interval, 2)
                    # else:
                    #     velocity = 0.00

                    # 创建关系，并附加距离、时间差（秒）和速度（米/秒）属性
                    relationship = Relationship(prev_node, "next", trajectory_node)#distance=distance, time_interval=time_interval, velocity=velocity
                    temp_relationship_ls.append(relationship)

            # 使用锁来确保多线程环境下对共享资源的安全访问
            with lock:
                trajectory_point_ls.extend(temp_trajectory_point_ls)
                relationship_ls.extend(temp_relationship_ls)

    for file_path in dir_xls:

        df_each = pd.read_csv(file_path, usecols=[0, 1, 2, 3, 4],
                              names=['vehicle', 'passenger_', 'datetime', 'longitude', 'latitude'], header=None)

        # 将 'longitude' 和 'latitude' 列转换为几何数据，并创建 GeoDataFrame
        trajectory = gpd.GeoDataFrame(df_each,
                                      geometry=gpd.points_from_xy(df_each['longitude'], df_each['latitude']),  # 生成几何点
                                      crs='EPSG:4326')  # 设置坐标系，使用WGS84坐标系
        trajectory = trajectory.to_crs(4549)

        # 添加 'pre_code_helper' 列
        trajectory['pre_code_helper'] = range(len(trajectory))

        # 计算trajectory的空间范围
        minx, miny, maxx, maxy = trajectory.total_bounds
    
        try:
            # 根据trajectory的空间范围裁剪road_gdf
            road_gdf_clipped = road_gdf.cx[minx:maxx, miny:maxy]

        except GEOSException as e:
            # 捕获 GEOSException 并跳过该次循环，打印出导致问题的文件路径
            print(f"GEOSException encountered in file {file_path}: {e}, skipping this iteration.")
            continue  # 跳过该次循环
    
        # 使用 spatial join 将轨迹点与 road_gdf_clipped 匹配
        trajectory_matched = gpd.sjoin_nearest(trajectory, road_gdf_clipped, how='inner', max_distance=30, distance_col="distance")

        if not trajectory_matched.empty:

            # 数据清洗：去除 'pre_code_helper' 列中重复的行，保留第一次出现的记录
            trajectory_matched = trajectory_matched.drop_duplicates(subset=['pre_code_helper'], keep='first')

            # 轨迹预处理：添加'datetime_helper'列，并将'datetime'转换为datetime类型
            trajectory_matched['datetime_helper'] = pd.to_datetime(trajectory_matched['datetime'], format="%Y-%m-%d %H:%M:%S")

            # 创建索引
            trajectory_matched = trajectory_matched.set_index('datetime_helper').sort_index()

            # 添加 'pre_code' 列
            trajectory_matched['pre_code'] = range(len(trajectory_matched))

            # 提取第一个 'vehicle' 的值
            vehicle = trajectory_matched['vehicle'].iat[0]  # 使用 .iat 进行更快速的数据访问

            start, end = 0, 1
            batch_num = 1

            # 使用 ThreadPoolExecutor 来并行处理时间切片
            with ThreadPoolExecutor(max_workers=120) as executor:  # 修改 max_workers 为 24
                futures = []
                for k in range(len(helper) - 1):
                    
                    # if batch_num == 2:
                    #     break

                    futures.append(executor.submit(process_slice, trajectory_matched, helper, start, end, batch_num))

                    start += 1
                    end += 1
                    batch_num += 1

                    # 每处理完一个时间切片，更新外部进度条
                    time_slice_pbar.update(1)

                # 等待所有并行任务完成
                for future in futures:
                    future.result()

                # 批量提交所有节点和关系到数据库
                subgraph = Subgraph(nodes=trajectory_point_ls, relationships=relationship_ls)
                tx = g.begin()
                tx.create(subgraph)
                g.commit(tx)

                # 清空临时列表
                trajectory_point_ls.clear()
                relationship_ls.clear()
                


Processing time slices:   0%|          | 0/119473920 [00:00<?, ?it/s]

GEOSException encountered in file F:\paper2\出租车原始数据\part1\1349.csv: IllegalArgumentException: Points of LinearRing do not form a closed linestring, skipping this iteration.
GEOSException encountered in file F:\paper2\出租车原始数据\part1\1435.csv: IllegalArgumentException: Points of LinearRing do not form a closed linestring, skipping this iteration.
GEOSException encountered in file F:\paper2\出租车原始数据\part1\2407.csv: IllegalArgumentException: Points of LinearRing do not form a closed linestring, skipping this iteration.
GEOSException encountered in file F:\paper2\出租车原始数据\part1\3229.csv: IllegalArgumentException: Points of LinearRing do not form a closed linestring, skipping this iteration.
GEOSException encountered in file F:\paper2\出租车原始数据\part1\4370.csv: IllegalArgumentException: Points of LinearRing do not form a closed linestring, skipping this iteration.
GEOSException encountered in file F:\paper2\出租车原始数据\part1\7402.csv: IllegalArgumentException: Points of LinearRing do not form a closed

ConnectionBroken: Failed to read message